# Box Loader Testing

I need to test a few things about the steps in the `argo_box_loader` notebook. There are a few things that might make a difference by decreasing the size of an array, but I want to try here on a box that takes a lot less time to load.

In [1]:
# PROCESSING FUNCTIONS

import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import argopy
import scipy.ndimage as filter
import scipy
import matplotlib
import gsw

import argopy
from argopy import DataFetcher as ArgoDataFetcher

argo_loader = ArgoDataFetcher(
    src="gdac", ftp="/swot/SUM05/dbalwada/Argo_sync", progress=True
)

In [22]:
import os
import sys
os.chdir('/home.ufs/amf2288/argo-intern/funcs')
sys.path.append(os.path.abspath('../funcs'))

import processing_funcs as prf

In [2]:
box = [-41,-40,0,1,0,2001]

In [3]:
ds = argo_loader.region(box)

In [9]:
ds

<datafetcher.gdac>
🌐 Name: Ifremer GDAC Argo data fetcher for a space/time region
🗺  Domain: [x=-41.00/-40.00; y=0.00/1.00; z=0.0/2001.0]
🔗 API: https://data-argo.ifremer.fr
📗 Index: ar_index_global_prof.txt.gz (3081434 records)
📸 Index searched: True (21 matches, 0.0007%)
🏊 User mode: standard
🟡+🔵 Dataset: phy
🌥  Performances: cache=False, parallel=False

In [7]:
type(ds)

argopy.fetchers.ArgoDataFetcher

In [8]:
ds.to_xarray()

100%|██████████| 21/21 [01:23<00:00,  3.99s/it]


<xarray.Dataset> Size: 2MB
Dimensions:          (N_POINTS: 22869)
Coordinates:
  * N_POINTS         (N_POINTS) int64 183kB 0 1 2 3 ... 22865 22866 22867 22868
    TIME             (N_POINTS) datetime64[ns] 183kB 1997-08-04T06:10:56 ... ...
    LATITUDE         (N_POINTS) float64 183kB 0.4 0.4 0.4 ... 0.02323 0.02323
    LONGITUDE        (N_POINTS) float64 183kB -40.16 -40.16 ... -40.22 -40.22
Data variables:
    CYCLE_NUMBER     (N_POINTS) int64 183kB 1 1 1 1 1 1 ... 114 114 114 114 114
    DATA_MODE        (N_POINTS) <U1 91kB 'R' 'R' 'R' 'R' 'R' ... 'R' 'R' 'R' 'R'
    DIRECTION        (N_POINTS) <U1 91kB 'A' 'A' 'A' 'A' 'A' ... 'A' 'A' 'A' 'A'
    PLATFORM_NUMBER  (N_POINTS) int64 183kB 31858 31858 ... 4903376 4903376
    POSITION_QC      (N_POINTS) int64 183kB 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
    PRES             (N_POINTS) float32 91kB 18.6 23.7 28.8 ... 1.998e+03 2e+03
    PRES_ERROR       (N_POINTS) float32 91kB nan nan nan nan ... nan nan nan nan
    PRES_QC          (N_POINTS) int64 183kB 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
    TEMP             (N_POINTS) float32 91kB 26.43 26.43 26.43 ... 3.646 3.644
    TEMP_ERROR       (N_POINTS) float32 91kB nan nan nan nan ... nan nan nan nan
    TEMP_QC          (N_POINTS) int64 183kB 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
    TIME_QC          (N_POINTS) int64 183kB 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
Attributes:
    raw_attrs:            {'raw_attrs': {'title': 'Argo float vertical profil...
    DATA_ID:              ARGO
    DOI:                  http://doi.org/10.17882/42182
    Fetched_from:         https://data-argo.ifremer.fr
    Fetched_by:           amf2288
    Fetched_date:         2024/12/28
    Fetched_constraints:  [x=-41.00/-40.00; y=0.00/1.00; z=0.0/2001.0]
    Fetched_uri:          https://data-argo.ifremer.fr/dac/aoml/1901814/19018...
    Processing_history:   [PRES,TEMP] real-time and adjusted/delayed variable...

Okay, this isn't incredibly helpful. Because it means I couldn't change the dtype of the data until after calling .to_xarray(), which is the step that uses up available memory. I'm now going to try loading multiple boxes in parallel using dask. Wish me luck

In [31]:
box1 = [-25,-24,0,1,0,2001]
box2 = [-25,-24,1,2,0,2001]
box3 = [-25,-24,2,3,0,2001]
box4 = [-25,-24,3,4,0,2001]
box5 = [-25,-24,4,5,0,2001]

In [32]:
import dask
from dask import delayed
from dask.distributed import Client

# Initialize the Dask client
client = Client()

# Define your function with the delayed decorator
@dask.delayed
def get_box_delayed(*args, **kwargs):
    return prf.get_box(*args, **kwargs)

# Example arguments for your function
list_of_args = [
    (box1,2),
    (box2,2),
    (box3,2),
    (box4,2),
    (box5,2)
]

# Create a list of delayed tasks
tasks = [get_box_delayed(*args) for args in list_of_args]

# Compute the results in parallel
results = dask.compute(*tasks)

# Process the results as needed
for n,result in enumerate(results):
    result.to_netcdf("/swot/SUM05/amf2288/sync-boxes/quads/lon:({},{})_lat:({},{})_ds_z.nc".format(list_of_args[n][0],list_of_args[n][1],list_of_args[n][2],list_of_args[n][3]))

/home.ufs/amf2288/mambaforge-pypy3/envs/Argo_Nov_24/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 38052 instead
  warnings.warn(


loading points complete
loading points complete
loading points complete
loading points complete


  0%|          | 0/43 [00:00<?, ?it/s]/home.ufs/amf2288/mambaforge-pypy3/envs/Argo_Nov_24/lib/python3.12/site-packages/argopy/data_fetchers/gdac_data.py:331: UserWarning: Found more than 50 files to load, this may take a while to process sequentially ! Consider using another data source (eg: 'erddap') or the 'parallel=True' option to improve processing time.
  warnings.warn(
  2%|▏         | 1/43 [00:01<00:57,  1.37s/it]

loading points complete


100%|██████████| 35/35 [01:58<00:00,  3.38s/it]
2024-12-28 15:47:51,730 - distributed.worker - ERROR - Compute Failed
Key:       get_box_delayed-bada1f6e-492b-46ec-a8ba-e1845ffd4a23
State:     executing
Function:  get_box_delayed
args:      ([-25, -24, 2, 3, 0, 2001], 2)
kwargs:    {}
Exception: 'KeyError("No variable named \'PSAL\'. Variables on the dataset include [\'CYCLE_NUMBER\', \'DATA_MODE\', \'DIRECTION\', \'PLATFORM_NUMBER\', \'POSITION_QC\', ..., \'TIME_QC\', \'N_POINTS\', \'TIME\', \'LATITUDE\', \'LONGITUDE\']")'
Traceback: '  File "/tmp/ipykernel_56565/1157822358.py", line 11, in get_box_delayed\n  File "/home/amf2288/argo-intern/funcs/processing_funcs.py", line 59, in get_box\n    ds = ds.argo.teos10(["CT", "SA", "SIG0"])\n         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^\n  File "/home.ufs/amf2288/mambaforge-pypy3/envs/Argo_Nov_24/lib/python3.12/site-packages/argopy/xarray.py", line 1366, in teos10\n    psal = this["PSAL"].values\n           ~~~~^^^^^^^^\n  File "/home.ufs/am

KeyError: "No variable named 'PSAL'. Variables on the dataset include ['CYCLE_NUMBER', 'DATA_MODE', 'DIRECTION', 'PLATFORM_NUMBER', 'POSITION_QC', ..., 'TIME_QC', 'N_POINTS', 'TIME', 'LATITUDE', 'LONGITUDE']"

100%|██████████| 52/52 [03:22<00:00,  3.90s/it]


In [35]:
cleint.close()

NameError: name 'cleint' is not defined